### Testing SQL Querying Using Langchain

In [6]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser, JsonKeyOutputFunctionsParser
from langchain.chat_models import ChatOllama
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
import json
from IPython.display import display, Markdown, Latex

In [3]:
os.environ["OPENAI_API_KEY"] = ""

#### Connect to DB using Langchain and test dummy questions

In [11]:
openai_llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

db = SQLDatabase.from_uri("sqlite:///DBs/Chinook.db")

sql_chain = create_sql_query_chain(llm=openai_llm, db=db, k=5)

In [12]:
sql_query = sql_chain.invoke(
    {
        "question": "How many employees are in the database?"
    }
)

In [13]:
sql_query

'SELECT COUNT(*) FROM Employee'